In [33]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
import tensorflow as tf
import pandas as pd
from utils.load import load_encoder_methodology
import numpy as np
import einops

ModuleNotFoundError: No module named 'einops'

In [2]:
df = pd.read_csv("../Metadata/data_curated.csv")
df.set_index("File", inplace=True)
df.head()

,Class
File,
S1V3C1M3A Clip166.npy,1
S1V1C1M1A Clip18.npy,1
S1V4C1M4A Clip241.npy,1
S1V4C1M4A Clip239.npy,1
S1V4C1M4A Clip245.npy,1


In [4]:
X_train, X_val = load_encoder_methodology(df,"M1")

In [26]:
X = []
for i in X_train:
    for j in range(i.shape[0]):
       X.append(i[j,:])

In [29]:
X = np.array(X)

In [31]:
class ResizeVideo(keras.layers.Layer):
  def __init__(self, height, width):
    super().__init__()
    self.height = height
    self.width = width
    self.resizing_layer = layers.Resizing(self.height, self.width)

  def call(self, video):
    """
      Use the einops library to resize the tensor.  

      Args:
        video: Tensor representation of the video, in the form of a set of frames.

      Return:
        A downsampled size of the video according to the new height and width it should be resized to.
    """
    # b stands for batch size, t stands for time, h stands for height, 
    # w stands for width, and c stands for the number of channels.
    old_shape = einops.parse_shape(video, 'b t h w c')
    images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
    images = self.resizing_layer(images)
    videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
    return videos


In [32]:
model = get_model(input_dim=X.shape[1], code_size=50)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 1662)]            0         
                                                                 
 encoder (Dense)             (None, 50)                83150     
                                                                 
 decoder (Dense)             (None, 1662)              84762     
                                                                 
Total params: 167912 (655.91 KB)
Trainable params: 167912 (655.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
